### Give the input here! 

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
### Instagram Username and Password
IG_username = os.getenv("username") ## -- ENTER USERNAME
IG_password = os.getenv("password") ## -- ENTER PASSWORD

chromedriver_path = os.getenv("chromedriver_path") ## -- UPDATE CHROME DRIVER PATH

### Give profile name of user as string ###Give Hashtags with '#' before the string

num_posts = int(os.getenv("num_posts"))

profiles_hashtags = ["#food"]
textfile_name = os.getenv("textfile") ## Enter text file name, that saves all image URLs. [Multiple files are created with name format]
folder_name = os.getenv("folder") ## Folder Name to save all images - Creates a folder if not available.

### Folder with images, and txt files with image URLs get saved in root path of this file

In [2]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
import time
import string
import random
# import tqdm.notebook as tq
import wget
import re

In [3]:
driver = webdriver.Chrome(chromedriver_path)
driver.maximize_window()
driver.get("http://www.instagram.com")

username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

username.clear()
username.send_keys(IG_username)
time.sleep(5)
password.clear()
password.send_keys(IG_password)
time.sleep(5)

button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

#Not now and Cancel buttons
not_now = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/section/main/div/div/div/div/button'))).click()
time.sleep(5)
not_now2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click()

/tmp/ipykernel_20370/1039144284.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_path)


### Scrapes image URLs from profiles and Hashtags mentioned, to text files

In [16]:
images_all = []
counts = 1
images_count = 0

textfile_path = textfile_name + ".txt"

for k in profiles_hashtags:
    if('#' in k):
        url = "https://www.instagram.com/explore/tags/" + k[1:]
    else:
        url = "https://www.instagram.com/" + k + "/"
    
    print(url)
    driver.get(url)
    time.sleep(5)
    
    posts = driver.find_element_by_xpath("(//span[@class='g47SY '])[1]")
    # num_posts = int(float(re.sub('\D', '', posts.text))/8)
    
    for i in range(num_posts):
        try:
            images = driver.find_elements_by_tag_name('img')
            for image in images: 
                images_all.append(image.get_attribute('src'))
        except Exception as e1:
            e1

        try:
            body = driver.find_element_by_css_selector('body')
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(5)
        except Exception as e:
            e
        print("Collected %d image"%(i+1))

        # if(images_count>=500):
        #     with open(textfile_path, "a") as outfile:
        #         outfile.write("\n".join(images_all))
        #         outfile.close()
        #     images_all = [" "]
        #     counts += 1
        #     textfile_path = textfile_name + str(counts) + ".txt"
        # elif(i >= num_posts):
        #     with open(textfile_path, "a") as outfile:
        #         outfile.write("\n".join(images_all))
        #         outfile.close()
        #     images_all = [" "]
        #     counts += 1
        #     textfile_path = textfile_name + str(counts) + ".txt"
    print("Completed")
    with open(textfile_path, "a") as outfile:
        outfile.write("\n".join(images_all))
        outfile.close()


driver.close()
driver.quit()

https://www.instagram.com/explore/tags/food


/tmp/ipykernel_33505/2697682312.py:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  posts = driver.find_element_by_xpath("(//span[@class='g47SY '])[1]")
/tmp/ipykernel_33505/2697682312.py:22: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  images = driver.find_elements_by_tag_name('img')
/tmp/ipykernel_33505/2697682312.py:29: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body = driver.find_element_by_css_selector('body')


Collected 1 image
Collected 2 image
Collected 3 image
Collected 4 image
Collected 5 image
Collected 6 image
Collected 7 image
Collected 8 image
Collected 9 image
Collected 10 image
Collected 11 image
Collected 12 image
Collected 13 image
Collected 14 image
Collected 15 image
Collected 16 image
Collected 17 image
Collected 18 image
Collected 19 image
Collected 20 image
Completed


### Saves images to folder

In [17]:
all_content = []
path_folder = os.getcwd()
path_folder = os.path.join(path_folder, folder_name)
if not os.path.exists(path_folder):
    os.mkdir(path_folder)
for i in range(1, counts+1):
    file_path = textfile_name + str(i) + ".txt"
    if(os.path.exists(file_path)):
        my_file = open(file_path, "r")
        content = my_file.read()
        content_list = content.split("\n")
        content_list = list(set(content_list))
        all_content.append(content_list)
        my_file.close()

from itertools import chain
flatten_list = list(chain.from_iterable(all_content))
flatten_list = list(set(flatten_list))
i=1
for image in flatten_list:
    res = ''.join(random.choices(string.ascii_uppercase + string.digits + string.ascii_lowercase, k = 15))
    save_as = os.path.join(path_folder, res + '.jpg')
    try:
        wget.download(image, save_as)
        if(os.path.getsize(save_as) < 10000):
            os.remove(save_as)
    except Exception as e2:
        print(image)
        print(e2)
print("\nCompleted")

100% [..............................................................................] 75672 / 75672https://instagram.fbdo4-1.fna.fbcdn.net/v/t51.2885-19/44884218_345707102882519_2446069589734326272_n.jpg?efg=eyJybWQiOiJpZ19hbmRyb2lkX21vYmlsZV9uZXR3b3JrX3N0YWNrX3N5c3RlbV90Y3BfZmFsbGJhY2tfd2l0aF9wb3N0XzI6c3lzdGVtX3RjcF9mYWxsYmFjayJ9&_nc_ht=instagram.fbdo4-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=lpqfWso6P6oAX8Kypjr&edm=AEaYFD0BAAAA&ccb=7-5&ig_cache_key=YW5vbnltb3VzX3Byb2ZpbGVfcGlj.2-ccb7-5&oh=00_AT_GpcWEs4vh-AKnwPUfoCdnPjlWOY6Bq-T-eqXDD_ulJw&oe=6291858F&_nc_sid=751ab8https://instagram.fmaa1-1.fna.fbcdn.net/v/t51.2885-15/282239380_147914184452843_5887182757747535817_n.jpg?stp=dst-webp_e35&cb=9ad74b5e-be52112b&_nc_ht=instagram.fmaa1-1.fna.fbcdn.net&_nc_cat=111&_nc_ohc=QRpK5AMde8kAX-srh01&edm=ABZsPhsBAAAA&ccb=7-5&ig_cache_key=Mjg0Mzc4MjM4NjYwNTU3NjI0Nw%3D%3D.2-ccb7-5&oh=00_AT-6yTLTjstiaWQ8OLH5F4NMuHwaLzLihKOB2jhkNDVDBw&oe=6290AEC8&_nc_sid=4efc9f
HTTP Error 403: Forbidden
100% [....................